In [10]:
import pandas as pd
import geopandas as gpd
import os
import requests
from bs4 import BeautifulSoup
from homeless import *
import openpyxl
from win32com.client import Dispatch
import numpy as np

In [11]:
outpath = r'G:\projects\UtilityDistricts\eweb\DrinkingWater\IllegalCampCoordination\Recieved'
path = outpath + '\\IllegalCampNotification_pro'

dat = pd.read_excel(path+'\\most_recent.xlsx')
cols2drop = ['OBJECTID', 'Join_Count', 'Unruly_inhabitants']
dat = dat.drop(cols2drop, axis=1)
d

In [13]:
dat = pd.read_excel(path+'\\most_recent.xlsx')
cols2drop = ['OBJECTID', 'Join_Count', 'Unruly_inhabitants']
dat = dat.drop(cols2drop, axis=1)
dat.columns = list(map(lambda x: x.capitalize(), dat.columns))
dat.rename(columns={'Ownname': 'Owner_name', 'Addr1': 'Owner_address'}, inplace=True)

In [20]:
dat = dat[['Target_fid', 'Status', 'Comments', 'Date', 'Submitted_by','Dogs_present', 'Hazardous_materials_present', 
'Biohazards_present','Size_of_encampment', 'Maptaxlot_hyphen', 'Owner_name', 'Owner_address', 'Nearby_owner', 
'Nearby_owner_address', 'Ownercity', 'Ownerprvst', 'Ownerzip', 'Geocity_name', 'Ugb_name','Longitude', 'Latitude']]

In [21]:
dat.rename(columns={'Target_fid': 'Target_FID'}, inplace=True)

In [22]:
taxlotcodes = pd.read_csv('mythical_taxlot_codes.csv')
for idx in range(0, dat.shape[0]):
    taxlotcode = int(dat.loc[idx,'Maptaxlot_hyphen'][-2:])
    if np.isnan(dat.loc[idx, 'Owner_name']) & (taxlotcode in taxlotcodes.end_number.values):
        dat.loc[idx, 'Owner_name'] = 'Mythical lot number for ' + taxlotcodes.loc[taxlotcodes.end_number == taxlotcode, 'taxlot'].values[0]
    
intakepath = r'G:\projects\UtilityDistricts\eweb\DrinkingWater\RiparianEcosystemMarketplace\market_area\REM_area.gdb'
intake_areas = gpd.read_file(intakepath, driver='FileGDB', layer='AboveIntake')
points = gpd.read_file(path + '\\MyProject4.gdb', driver='FileGDB', layer='HomelessCampSite_SpatialJoin')
points = points.to_crs(epsg=2914)
print("Read data...")

urlstart ='https://services5.arcgis.com/9s1YtFmLS0YTl10F/arcgis/rest/services/ZHomeless_Camp_Trash_Collector/FeatureServer/0/'
urlend = '/attachments?f=html&token='

datestr = points.Date.values[0].split('T')[0]
res = convert_date(datestr)
Y = res[1]
m = res[2]
d = res[3]
outfolder = os.path.join(outpath, Y, m+'_'+d)
filename = 'IllegalCampNotice_'+m+'_'+d+'_'+Y[2:4]+'.xlsx'
file = os.path.join(outfolder, filename)
print("Got file name...")

Read data...
Got file name...


In [23]:
for pID in points.index:
    point = points[points.index==pID]
    if all(intake_areas.contains(point)):
        dat.loc[pID, 'Above_Intake'] = 'Yes'
    else:
        dat.loc[pID, 'Above_Intake'] = 'No'
    FID = dat.loc[pID, 'Target_FID']
    url= urlstart+str(FID)+urlend
    resp=requests.get(url)
    links = []
    if resp.status_code==200:
        soup=BeautifulSoup(resp.text,'html.parser')
        for link in soup.findAll('a'):
            links.append(link.get('href'))
    else:
        print("Error")
    attached = [link for link in links if 'attachments' in link]
    if len(attached)==0:
        dat.loc[pID, 'Photos'] = 'NA'
    elif len(attached)==1:
        dat.loc[pID, 'Photos'] = 'https://services5.arcgis.com' + attached[0]
    else:
        dat.loc[pID, 'Photos'] = '; '.join(['https://services5.arcgis.com' + s for s in attached])

print("Edited data...")

dat.loc[:, 'Date'] = dat.Date.astype(str)
dat.to_excel(file, index=False)
print("Exported data...")


C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\base.py:32: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


Edited data...
Exported data...


In [24]:
dat = pd.read_excel(file)
wb = openpyxl.load_workbook(file)
ws = wb.active
ws = removeFormatting(ws)
for pID in points.index:
    photovalue = dat.loc[pID, 'Photos']
    if photovalue == 'NA':
        print(f"No photos at Point {dat.loc[pID, 'Target_FID']}")
    else:
        ws.cell(row=2, column=21).value = 'Yes'
        if ';' in photovalue: 
            urls = photovalue.split('; ')
            for i in range(0, len(urls)):
                ws.cell(row=2, column=22+i).value = '=HYPERLINK("{}", "{}")'.format(urls[i],'Photo '+str(i+1))
                ws.cell(row=2, column=22+i).style = "Hyperlink"
        else:
            ws.cell(row=2, column=22).value = '=HYPERLINK("{}", "{}")'.format(photovalue,'Photo')
            ws.cell(row=2, column=22).style = "Hyperlink"
wb.save(file)
print("Checked photos...")
excel = Dispatch('Excel.Application')
wb = excel.Workbooks.Open(file)
excel.Worksheets(1).Activate()
excel.ActiveSheet.Columns.AutoFit()
wb.Close(True)
print("Autofitted columns...")

Checked photos...
Autofitted columns...


In [5]:
for pID in points.index:
    point = points[points.index==pID]
    if all(intake_areas.contains(point)):
        dat.loc[pID, 'Above_Intake'] = 'Yes'
    else:
        dat.loc[pID, 'Above_Intake'] = 'No'
    FID = dat.loc[pID, 'Target_FID']
    url= urlstart+str(FID)+urlend
    resp=requests.get(url)
    links = []
    if resp.status_code==200:
        soup=BeautifulSoup(resp.text,'html.parser')
        for link in soup.findAll('a'):
            links.append(link.get('href'))
    else:
        print("Error")
    attached = [link for link in links if 'attachments' in link]
    if len(attached)==0:
        dat.loc[pID, 'Photos'] = 'NA'
    elif len(attached)==1:
        dat.loc[pID, 'Photos'] = 'https://services5.arcgis.com' + attached[0]
    else:
        dat.loc[pID, 'Photos'] = '; '.join(['https://services5.arcgis.com' + s for s in attached])

print("Edited data...")

dat.loc[:, 'Date'] = dat.Date.astype(str)
dat.to_excel(file, index=False)
print("Exported data...")

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\base.py:32: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")


Edited data...


PermissionError: [Errno 13] Permission denied: 'G:\\projects\\UtilityDistricts\\eweb\\DrinkingWater\\IllegalCampCoordination\\Recieved\\2023\\1_26\\IllegalCampNotice_1_26_23.xlsx'

In [3]:
s = 'TARGET_FID'

In [9]:
dat.rename(columns={'Ownname': 'Own_name', 'Addr1': 'Owner_address'}, inplace=True)

In [8]:
dat.columns = list(map(lambda x: x.capitalize(), dat.columns))

In [10]:
dat.rename(columns={'Ownname': 'Owner_name', 'Addr1': 'Owner_address'}, inplace=True)

In [22]:
taxlotcodes = pd.read_csv('mythical_taxlot_codes.csv')

In [35]:
(dat.Owner_name.isnull()) & (int(dat.Maptaxlot_hyphen[0][-2:]) in taxlotcodes.end_number.values)

0    True
Name: Owner_name, dtype: bool

In [37]:
dat.columns = ['Target_fid', 'Status', 'Comments', 'Date', 'Submitted_by',
       'Dogs_present', 'Hazardous_materials_present', 'Biohazards_present',
       'Size_of_encampment', 'Maptaxlot_hyphen', 'Owner_name', 'Owner_address', 
               'Nearby_owner', 'Nearby_owner_address', 
       'Ownercity', 'Ownerprvst', 'Ownerzip', 'Geocity_name', 'Ugb_name',
       'Longitude', 'Latitude']

In [38]:
idx = 0

In [45]:
dat.loc[idx,'Maptaxlot_hyphen'][-2:]

'77'

In [1]:
import geopandas as gpd

C:\Users\clid1852\AppData\Anaconda3\envs\geoenv\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
import pandas as pd

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
import xlsxwriter

In [88]:
from win32com.client import Dispatch

In [5]:
path = r'G:\projects\UtilityDistricts\eweb\DrinkingWater\IllegalCampCoordination\Recieved\IllegalCampNotification_pro'

In [3]:
dat = pd.read_excel(path+'\\most_recent.xlsx')

In [5]:
dat.columns

Index(['OBJECTID', 'Join_Count', 'TARGET_FID', 'Status', 'Comments', 'Date',
       'Submitted_by', 'Dogs_present', 'Unruly_inhabitants',
       'Hazardous_materials_present', 'Biohazards_present',
       'Size_of_encampment', 'maptaxlot_hyphen', 'ownname', 'addr1',
       'ownercity', 'ownerprvst', 'ownerzip', 'geocity_name', 'ugb_name',
       'longitude', 'latitude'],
      dtype='object')

In [6]:
cols2drop = ['OBJECTID', 'Join_Count', 'Unruly_inhabitants']

In [7]:
dat = dat.drop(cols2drop, axis=1)

In [8]:
intakepath = r'G:\projects\UtilityDistricts\eweb\DrinkingWater\RiparianEcosystemMarketplace\market_area\REM_area.gdb'

In [10]:
intake_areas = gpd.read_file(intakepath, driver='FileGDB', layer='AboveIntake')

In [11]:
intake_areas.crs

<Projected CRS: EPSG:2914>
Name: NAD83(HARN) / Oregon South (ft)
Axis Info [cartesian]:
- X[east]: Easting (foot)
- Y[north]: Northing (foot)
Area of Use:
- name: United States (USA) - Oregon - counties of Coos; Crook; Curry; Deschutes; Douglas; Harney; Jackson; Josephine; Klamath; Lake; Lane; Malheur.
- bounds: (-124.6, 41.98, -116.9, 44.56)
Coordinate Operation:
- name: SPCS83 Oregon South zone (International feet)
- method: Lambert Conic Conformal (2SP)
Datum: NAD83 (High Accuracy Reference Network)
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [12]:
point = gpd.read_file(path + '\\MyProject4.gdb', driver='FileGDB', layer='most_recent')

In [14]:
point = point.to_crs(epsg=2914)

In [20]:
dat.loc[:, 'TARGET_FID'] = dat.loc[:, 'TARGET_FID'] + 1085

Index(['TARGET_FID', 'Status', 'Comments', 'Date', 'Submitted_by',
       'Dogs_present', 'Hazardous_materials_present', 'Biohazards_present',
       'Size_of_encampment', 'maptaxlot_hyphen', 'ownname', 'addr1',
       'ownercity', 'ownerprvst', 'ownerzip', 'geocity_name', 'ugb_name',
       'longitude', 'latitude'],
      dtype='object')

In [48]:
point

,Status,Comments,Date,Submitted_by,Dogs_present,Unruly_inhabitants,Hazardous_materials_present,Biohazards_present,Size_of_encampment,Date_str,geometry
0,Occupied,Homeless camp,2023-01-18T22:52:35+00:00,Matthew Gilliam 541-556-1784,Unknown,None,Unknown,Unknown,Small,1/18/2023,POINT (4228098.619 897149.301)


In [61]:
url='https://services5.arcgis.com/9s1YtFmLS0YTl10F/arcgis/rest/services/ZHomeless_Camp_Trash_Collector/FeatureServer/0/1831/attachments?f=html&token='

In [62]:
#open with GET method
resp=requests.get(url)

In [64]:
if resp.status_code==200:
    soup=BeautifulSoup(resp.text,'html.parser')

In [66]:
links = []

In [67]:
for link in soup.findAll('a'):
    links.append(link.get('href'))

In [69]:
attached = [link for link in links if 'attachments' in link]

In [74]:
type(attached)

list

In [78]:
'; '.join(['https://services5.arcgis.com' + s for s in attached])

'https://services5.arcgis.com/9s1YtFmLS0YTl10F/ArcGIS/rest/services/ZHomeless_Camp_Trash_Collector/FeatureServer/0/1831/attachments/559; https://services5.arcgis.com/9s1YtFmLS0YTl10F/ArcGIS/rest/services/ZHomeless_Camp_Trash_Collector/FeatureServer/0/1831/attachments/560; https://services5.arcgis.com/9s1YtFmLS0YTl10F/ArcGIS/rest/services/ZHomeless_Camp_Trash_Collector/FeatureServer/0/1831/attachments/561'

In [85]:
datestr = points.Date.values[0].split('T')[0]

In [86]:
def convert_date(datestr):
    datestrlist = datestr.split('-')
    Y = datestrlist[0]
    m = str(int(datestrlist[1]))
    d = str(int(datestrlist[2]))
    res = m+'/'+d+'/'+Y
    return res, Y, m, d

In [4]:
gdat = gpd.read_file(path + '\\MyProject4.gdb', driver='FileGDB', layer='datacopy')